<div >
<img src = "../../banner.jpg" />
</div>

<a target="_blank" href="https://colab.research.google.com/github/ignaciomsarmiento/BDML_SS/blob/main/Lecture07/Notebook_SS0.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Regularization: Lasso

## Predicting Wages

Our objective today is to construct a model of individual wages

$$
w = f(X) + u 
$$

where w is the  wage, and X is a matrix that includes potential explanatory variables/predictors. In this problem set, we will focus on a linear model of the form

\begin{align}
 ln(w) & = \beta_0 + \beta_1 X_1 + \dots + \beta_p X_p  + u 
\end{align}

were $ln(w)$ is the logarithm of the wage.

To illustrate I'm going to use a sample of the NLSY97. The NLSY97 is  a nationally representative sample of 8,984 men and women born during the years 1980 through 1984 and living in the United States at the time of the initial survey in 1997.  Participants were ages 12 to 16 as of December 31, 1996.  Interviews were conducted annually from 1997 to 2011 and biennially since then.  

Let's load the packages and the data set:

In [ ]:
# install.packages("pacman") #run this line if you use Google Colab

In [ ]:
#packages
require("pacman")
p_load("tidyverse","stargazer")

nlsy <- read_csv('https://raw.githubusercontent.com/ignaciomsarmiento/datasets/main/nlsy97.csv')

nlsy = nlsy  %>%   drop_na(educ) #dropea los valores faltantes (NA)

We want to construct a model that predicts well out of sample, and we have potentially 994 regressors. We are going to regularize this regression using Ridge.

## Lasso

We first illustrate ridge regression, which can be fit using glmnet() with alpha = 1 and seeks to minimize

$$
\sum_{i=1}^{n} \left( y_i - \beta_0 - \sum_{j=1}^{p} \beta_j x_{ij}    \right) ^ 2 + \lambda \sum_{j=1}^{p} |\beta_j|.
$$

Notice that the intercept is not penalized. 


We are going to use Glmnet. Glmnet is a package that fits generalized linear and similar models via penalized maximum likelihood. The regularization path is computed for the lasso or elastic net penalty at a grid of values (on the log scale) for the regularization parameter lambda. The algorithm is extremely fast!

## Intuition: few predictors

In [ ]:
p_load("glmnet")

In [ ]:
#Vector that needs predicting
y <- nlsy$lnw_2016

# Matrix of predictos (only educ and afqt)
X <- as.matrix(nlsy  %>% select(educ,mom_educ,dad_educ))



### No penalty = OLS

In [ ]:
lasso_no_pen <- glmnet(
  x = X,
  y = y,
  alpha = 1, #lasso
  lambda=0
)

In [ ]:
lasso_no_pen$beta

In [ ]:
summary(lm(y~X))

### With Penalty

In [ ]:
lasso_pen <- glmnet(
  x = X,
  y = y,
  alpha = 1, #lasso
  lambda=.2
)

In [ ]:
lasso_pen$beta

### Larger Penalty

In [ ]:
lasso_pen_large <- glmnet(
  x = X,
  y = y,
  alpha = 1, #lasso
  lambda=1e70
)

In [ ]:
lasso_pen_large$beta

In [ ]:
### Various Penalties

In [ ]:
lasso01 <- glmnet(
  x = X,
  y = y,
  alpha = 1 #lasso
)

In [ ]:

plot(lasso01, xvar = "lambda")

## Lasso all predictors

In [ ]:
# Matrix of predictos (all but lnw_2016)
X0 <- as.matrix(nlsy  %>% select(-lnw_2016))

#Vector that needs predicting
y <- nlsy$lnw_2016


lasso0 <- glmnet(
  x = X0,
  y = y,
  alpha = 1 #lasso
)


plot(lasso0, xvar = "lambda")

## Selección de la penalización

In [ ]:
p_load("caret")

In [ ]:
set.seed(123)
fitControl <- trainControl(## 5-fold CV, 10 better
                           method = "cv",
                           number = 5)

In [ ]:
lasso<-train(lnw_2016~.,
             data=nlsy,
             method = 'glmnet', 
             trControl = fitControl,
             tuneGrid = expand.grid(alpha = 1, #lasso
                                    lambda = lasso0$lambda)
              ) 


In [ ]:
plot(lasso$results$lambda,
     lasso$results$RMSE,
     xlab="lambda",
     ylab="Root Mean-Squared Error (RMSE)"
     )

In [ ]:
lasso$bestTune

In [ ]:
coef_lasso<-coef(lasso$finalModel, lasso$bestTune$lambda)
coef_lasso

### Compare to OLS fit

In [ ]:
lasso$results$RMSE[which.min(lasso$results$lambda)]

In [ ]:
linear_reg<-train(lnw_2016~.,
                 data=nlsy,
                  method = 'lm', 
                  trControl = fitControl
) 


linear_reg

### Compare to Ridge?

In [ ]:
ridge<-train(lnw_2016~.,
             data=nlsy,
             method = 'glmnet', 
             trControl = fitControl,
             tuneGrid = expand.grid(alpha = 0, #ridge
                                    lambda = lasso0$lambda)
              ) 


In [ ]:
RMSE_df<-cbind(linear_reg$results$RMSE,
               ridge$results$RMSE[which.min(ridge$results$lambda)],
               lasso$results$RMSE[which.min(lasso$results$lambda)]
              )
colnames(RMSE_df)<-c("OLS","RIDGE","LASSO")
RMSE_df

## Elastic Net

\begin{align}
min_{\beta} EN(\beta) &= \sum_{i=1}^n (y_i-\beta_0 - \sum_{j=1}^p x_{ij}\beta_j)^2  + \lambda\left(\alpha \sum_{j=1}^p |\beta_j| + \frac{(1-\alpha)}{2} \sum_{j=1}^p (\beta_j)^2\right)
\end{align}

In [ ]:
EN<-train(lnw_2016~.,
             data=nlsy,
             method = 'glmnet', 
             trControl = fitControl,
             tuneGrid = expand.grid(alpha =seq(0,1,0.1),
                                    lambda = lasso0$lambda)
              ) 

In [ ]:
EN$bestTune

### Compare

#### Coeficients

In [ ]:
coef_lasso<-coef(lasso$finalModel, lasso$bestTune$lambda)
coef_ridge<-coef(ridge$finalModel, ridge$bestTune$lambda)
coef_EN<-coef(EN$finalModel,EN$bestTune$lambda)

coefs_df<-cbind(coef(linear_reg$finalModel),as.matrix(coef_ridge),as.matrix(coef_lasso),as.matrix(coef_EN))
colnames(coefs_df)<-c("OLS","RIDGE","LASSO","ELASTIC_NET")
round(coefs_df,4)

#### RMSE

In [ ]:
RMSE_df<-cbind(linear_reg$results$RMSE,ridge$results$RMSE[which.min(ridge$results$lambda)],lasso$results$RMSE[which.min(lasso$results$lambda)],EN$results$RMSE[which.min(EN$results$lambda)])
colnames(RMSE_df)<-c("OLS","RIDGE","LASSO","EN")
RMSE_df